# cursor for database - Buffered and unBuffered

In [89]:
import mysql.connector

In [90]:
host_name = "localhost"
user_name = "root"
user_password = "root@123"
user_auth_plugin = "mysql_native_password"
database_name = "testing"
database_port = 3306

In [91]:
conn = mysql.connector.connect(
	host=host_name,
	user=user_name,
	passwd=user_password,
	auth_plugin=user_auth_plugin,
	db=database_name,
	port=database_port
	)

In [92]:
buffered_cursor = conn.cursor(buffered=True)
unbuffered_cursor = conn.cursor(buffered=False)


rowcount attribute of an unbuffered cursor returns -1 right after the execute method is called. This, basically, means that the entire result set has not yet been fetched from the server. Furthermore, the rowcount attribute of an unbuffered cursor increases as you fetch rows from it, while the rowcount attribute of a buffered cursor remains the same, as you fetch rows from it.

In [93]:
# Below code giving error when we are trying to execute multiple queries , Its known Bug so i am diving code in step by step to 
# avoid error

# create_query = """
# drop table if exists people;
# create table if not exists people (
#     personid int(10) unsigned auto_increment,
#     firstname varchar(255),
#     lastname varchar(255),
#     primary key (personid)
# );
# insert into people (firstname, lastname)
# values ('Jon', 'Bon Jovi'),
# ('David', 'Bryan'),
# ('Tico', 'Torres'),
# ('Phil', 'Xenidis'),
# ('Hugh', 'McDonald')
# """
# results = buffered_cursor.execute(create_query, multi=True)
# conn.commit()



In [94]:
create_query = "drop table if exists people"

In [95]:
# Create and populate a table
# results = buffered_cursor.execute(create_query,multi=True)
buffered_cursor.execute(create_query)


In [96]:
conn.commit()

In [97]:
create_query = """create table people ( 
    personid int(10) unsigned auto_increment,
    firstname varchar(255),
    lastname varchar(255),
    primary key (personid)
)"""

In [98]:
create_query

'create table people ( \n    personid int(10) unsigned auto_increment,\n    firstname varchar(255),\n    lastname varchar(255),\n    primary key (personid)\n)'

In [99]:
buffered_cursor.execute(create_query)

In [100]:
conn.commit()

In [101]:
create_query = """insert into people (firstname, lastname)
values ('Jon', 'Bon Jovi'),
('David', 'Bryan'),
('Tico', 'Torres'),
('Phil', 'Xenidis'),
('Hugh', 'McDonald')"""

In [102]:
create_query

"insert into people (firstname, lastname)\nvalues ('Jon', 'Bon Jovi'),\n('David', 'Bryan'),\n('Tico', 'Torres'),\n('Phil', 'Xenidis'),\n('Hugh', 'McDonald')"

In [103]:
buffered_cursor.execute(create_query)

In [104]:
conn.commit()

In [105]:
buffered_cursor.execute("select * from people")
print("Row count from a buffer cursor:", buffered_cursor.rowcount)
unbuffered_cursor.execute("select * from people")
print("Row count from an unbuffered cursor:", unbuffered_cursor.rowcount)


Row count from a buffer cursor: 5
Row count from an unbuffered cursor: -1


In [106]:
while True:
    try:
        row = next(buffered_cursor)
        print("Row:", row)
        print("Row count:", buffered_cursor.rowcount)
    except StopIteration:
        break

print()


Row: (1, 'Jon', 'Bon Jovi')
Row count: 5
Row: (2, 'David', 'Bryan')
Row count: 5
Row: (3, 'Tico', 'Torres')
Row count: 5
Row: (4, 'Phil', 'Xenidis')
Row count: 5
Row: (5, 'Hugh', 'McDonald')
Row count: 5



In [107]:
while True:
    try:
        row = next(unbuffered_cursor)
        print("Row:", row)
        print("Row count:", unbuffered_cursor.rowcount)
    except StopIteration:
        break

Row: (1, 'Jon', 'Bon Jovi')
Row count: 1
Row: (2, 'David', 'Bryan')
Row count: 2
Row: (3, 'Tico', 'Torres')
Row count: 3
Row: (4, 'Phil', 'Xenidis')
Row count: 4
Row: (5, 'Hugh', 'McDonald')
Row count: 5


Note the row  count for buffered_cursor is always 5 i.e iterator

row count for unbuffered_cursor is 1,2,3,4,5 i.e Generator.

In [109]:
conn.close()

Another way of showing difference

In [110]:
conn = mysql.connector.connect(
	host=host_name,
	user=user_name,
	passwd=user_password,
	auth_plugin=user_auth_plugin,
	db=database_name,
	port=database_port
	)
buffered_cursor = conn.cursor(buffered=True)
unbuffered_cursor = conn.cursor(buffered=False)


In [111]:
create_query = "drop table if exists people"
buffered_cursor.execute(create_query)
conn.commit()

create_query = """create table people ( 
    personid int(10) unsigned auto_increment,
    firstname varchar(255),
    lastname varchar(255),
    primary key (personid)
)"""
buffered_cursor.execute(create_query)
conn.commit()


create_query = """insert into people (firstname, lastname)
values ('Jon', 'Bon Jovi'),
('David', 'Bryan'),
('Tico', 'Torres'),
('Phil', 'Xenidis'),
('Hugh', 'McDonald')"""
buffered_cursor.execute(create_query)
conn.commit()


In [112]:
unbuffered_cursor.execute("select * from people")
unbuffered_cursor.fetchone()

(1, 'Jon', 'Bon Jovi')

In [113]:
buffered_cursor.execute("select * from people")

InternalError: Unread result found

The snippet above will raise a InternalError exception with a message indicating that there is some unread result. What it is basically saying is that the result returned by the unbuffered cursor needs to be fully consumed before you can execute another query with any cursor under the same connection. If you change unbuffered_cursor.fetchone() with unbuffered_cursor.fetchall(), the error will disappear.

In [114]:
conn.close()